In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
%matplotlib inline
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import xgboost
import re

from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline


### будем использовать в качестве метрики RMSE

In [ ]:
def rmse(x,y):
    return np.mean((x - y) ** 2) ** 0.5

### загрузка данных url_domain_train. Пока работаем только с ними

In [ ]:
urls_train_df = pd.read_csv('data/url_domain_train.csv', header=None, delimiter='\t')

In [ ]:
urls_train_df.columns = ['id', 'url', 'count']
urls_train_df.head()

### нужно учесть count

#### попробуем разбивать url'ы на части: login.rutracker.org $\to$ 'login rutracker org'

In [ ]:
sep_tmp = urls_train_df
sep_tmp['url'] = sep_tmp['url'].astype('str')
sep_tmp['space'] = ' '
sep_tmp['array_of_url'] = (sep_tmp['url'] + sep_tmp['space']).apply(lambda x: re.findall(r"[\w']+",x)) * sep_tmp['count']
sep_tmp.drop(['space'], axis = 1, inplace=True)
sep_tmp.head()

In [ ]:
sep_URLS = pd.DataFrame(sep_tmp.groupby('id').array_of_url.apply(lambda x: x.tolist()))
sep_URLS.head()

#### без разбиения самих url'ов

In [ ]:
tmp = urls_train_df
tmp['url'] = tmp['url'].astype('str')
tmp['space'] = ' '
tmp['array_of_url'] = (tmp['url'] + tmp['space']).apply(lambda x: x.split(' ')[:-1]) * tmp['count']
tmp.drop(['space'], axis = 1, inplace=True)
tmp.head()

In [ ]:
URLS = pd.DataFrame(tmp.groupby('id').array_of_url.apply(lambda x: x.tolist()))

In [ ]:
sep_URLS.head()

In [ ]:
URLS.head()

In [ ]:
def sep_f(x):
    a = []
    for i in range(len(x)):
        for j in range(len(x[i])):
            a.append(x[i][j])
    return " ".join(a)

In [ ]:
def f(x):
    a = []
    for i in range(len(x)):
        for j in range(len(x[i])):
            a.append(x[i][j])
    return a

In [ ]:
URLS['list'] = URLS.array_of_url.apply(f)
URLS.drop(['array_of_url'], axis = 1, inplace=True)

sep_URLS['list'] = sep_URLS.array_of_url.apply(sep_f)
sep_URLS.drop(['array_of_url'], axis = 1, inplace=True)

In [ ]:
URLS['id'] = URLS.index
URLS.index = range(len(URLS))
URLS.columns = ['urls', 'id']

sep_URLS['id'] = sep_URLS.index
sep_URLS.index = range(len(sep_URLS))
sep_URLS.columns = ['urls', 'id']

In [ ]:
age_train_df = pd.read_csv('data/age_profile_train.csv', header=None, delimiter='\t')
age_train_df.columns = ['id', 'age']
age_train_df.head()

In [ ]:
train_df = URLS.merge(age_train_df, on='id', how='left')
sep_train_df = sep_URLS.merge(age_train_df, on = 'id', how='left')

In [ ]:
train_df.head()

In [ ]:
sep_train_df.head()

### начнём делать кросс-валидацию и обработку признаков

In [ ]:
y = train_df.age.values

In [ ]:
train_df.urls = train_df.urls.apply(lambda x: " ".join(x))

In [ ]:
train_df.urls[:2]

In [ ]:
sep_train_df.urls[:2]

In [ ]:
tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
X = tfidf.fit_transform(train_df.urls.values)

tfidf = sklearn.feature_extraction.text.TfidfVectorizer()
sep_X = tfidf.fit_transform(sep_train_df.urls.values)

In [ ]:
print sep_X.shape
print X.shape

In [ ]:
score = []
sep_score = []
for k in range(5):
    mean = []
    sep_mean = []
    for i in range(5):

    
        #train_data, test_data, y_train, y_test = cross_validation.train_test_split(X_nmf, y, \
        #                                                             test_size = 0.3)
    
        #xgb_rg = xgboost.XGBRegressor(n_estimators=n, learning_rate=0.03)
        #xgb_rg.fit(train_data, y_train)
        #print 'xgboost_rmse = ', rmse(xgb_rg.predict(test_data), y_test)
        #mean.append(rmse(xgb_rg.predict(test_data), y_test))
        
        train_data, test_data, y_train, y_test = cross_validation.train_test_split(X, y,\
                                                                                  test_size = 0.3)
        lin_reg = linear_model.SGDRegressor(n_iter=650, alpha=0.000008)
        lin_reg.fit(train_data, y_train)
        print rmse(lin_reg.predict(test_data), y_test)
        mean.append(rmse(lin_reg.predict(test_data), y_test))
        
        train_data, test_data, y_train, y_test = cross_validation.train_test_split(sep_X, y,\
                                                                                  test_size = 0.3)
        lin_reg = linear_model.SGDRegressor(n_iter=650, alpha=0.000008)
        lin_reg.fit(train_data, y_train)
        print 'with sep urls ',rmse(lin_reg.predict(test_data), y_test)
        sep_mean.append(rmse(lin_reg.predict(test_data), y_test))
        
    score.append(np.mean(mean))
    sep_score.append(np.mean(sep_mean))
print np.mean(score)
print 'in general with sep =', np.mean(sep_score)
    

### с раздельными url'ами лучше!

In [ ]:
model = decomposition.NMF(n_components=200)
model.fit(sep_X) 

In [ ]:
reg = LinearRegression()
reg.fit(X_nmf[itrain, :], y[itrain])

pred_X_nmf = reg.predict(X_nmf[itest, :])
print rmse(y[itest], pred_X_nmf)

In [ ]:
error = []
for w in np.linspace(0, 1, 100):
     error += [rmse(y[itest], pred_X * w + (1-w) * pred_X_nmf)]

pd.Series(error, index=np.linspace(0, 1, 100)).plot(figsize=(6,3))
print min(error)

In [ ]:
from sklearn import pipeline, preprocessing, feature_extraction

In [ ]:
# обычный датасет
dataset1 = Dataset(X[itrain, :], y[itrain], X[itest, :])
# NMF
dataset2 = Dataset(X_nmf[itrain, :], y[itrain], X_nmf[itest, :])

# линейная модель на обычных данных
lr = Regressor(dataset=dataset1, 
                     estimator=linear_model.LinearRegression,
                     parameters={'normalize': True},
                     name='lr')

# линейная модель на TFIDF от обычных данных
def tfidf_lr_(X_train, y_train, X_test, y_test=None, random_state=8888):
    model = pipeline.Pipeline([('tfidf', feature_extraction.text.TfidfTransformer()), 
                                ('linear_model', linear_model.LinearRegression())])
    model.fit(X_train, y_train)
    return model.predict(X_test)

tfidf_lr = Regressor(dataset=dataset1,
                     estimator=tfidf_lr_,
                     name='tfidf_lr')

# ExtraTrees на NMF
rf = Regressor(dataset=dataset2, 
                     estimator=ensemble.ExtraTreesRegressor,
                     parameters={'n_estimators': 100, 'n_jobs': -1},
                     name='rf')

# Stack two models
# Returns new dataset with out-of-fold predictions
meta_pipeline = ModelsPipeline(lr, tfidf_lr, rf)
stack_ds = meta_pipeline.blend(proportion=0.2,seed=111)

# Train LinearRegression on stacked data (second stage)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
results = stacker.predict()

print rmse(y[itest], results)

### Теперь загрузим title_unify_train

In [ ]:
title_train = pd.read_csv('data/title_unify_train.csv', header=None, sep='\t')
title_train.head()

In [ ]:
title_train.columns = ['id', 'unify', 'count']

In [ ]:
tmp = title_train
tmp['unify'] = tmp['unify'].astype('str')
tmp['space'] = ' '
tmp['array_of_unify'] = (tmp['unify'] + tmp['space']).apply(lambda x: x.split(' ')[:-1]) * tmp['count']
tmp.drop(['space'], axis = 1, inplace=True)
tmp.head()

In [ ]:
UNIFY = pd.DataFrame(tmp.groupby('id').array_of_unify.apply(lambda x: x.tolist()))

In [ ]:
UNIFY['list'] = UNIFY.array_of_unify.apply(f)
UNIFY.drop(['array_of_unify'], axis = 1, inplace=True)

In [ ]:
UNIFY['id'] = UNIFY.index
UNIFY.index = range(len(UNIFY))
UNIFY.columns = ['unify', 'id']
UNIFY.head()

In [ ]:
train_un = UNIFY.merge(age_train_df, on='id', how='left')
train_un.head()

In [ ]:
X, y = train_un.unify.values, train_un.age.values

In [ ]:
X = map(lambda x: ' '.join(x), X)
hw = HashingVectorizer(n_features=1500, non_negative=True).fit(X)
X_to_den = hw.transform(X).todense()

In [ ]:
%%time

transformer = sklearn.decomposition.NMF(n_components=100)
X_nmf = transformer.fit_transform(X_to_den)

In [ ]:
itrain, itest = cross_validation.train_test_split(range(len(X)), test_size=1./3, random_state=0)
len(itrain), len(itest)

In [ ]:
rmse(y[itest], np.mean(y[itrain]))

In [ ]:
reg = LinearRegression()
reg.fit(X[itrain, :], y[itrain])

pred_X = reg.predict(X[itest, :])
print rmse(y[itest], pred_X)

In [ ]:
reg = LinearRegression()
reg.fit(X_nmf[itrain, :], y[itrain])

pred_X_nmf = reg.predict(X_nmf[itest, :])
print rmse(y[itest], pred_X_nmf)

In [ ]:
error = []
for w in np.linspace(0, 1, 100):
     error += [rmse(y[itest], pred_X * w + (1-w) * pred_X_nmf)]

pd.Series(error, index=np.linspace(0, 1, 100)).plot(figsize=(6,3))
print min(error)

In [ ]:
# обычный датасет
dataset1 = Dataset(X[itrain, :], y[itrain], X[itest, :])
# NMF
dataset2 = Dataset(X_nmf[itrain, :], y[itrain], X_nmf[itest, :])

# линейная модель на обычных данных
lr = Regressor(dataset=dataset1, 
                     estimator=linear_model.LinearRegression,
                     parameters={'normalize': True},
                     name='lr')

# линейная модель на TFIDF от обычных данных
def tfidf_lr_(X_train, y_train, X_test, y_test=None, random_state=8888):
    model = pipeline.Pipeline([('tfidf', feature_extraction.text.TfidfTransformer()), 
                                ('linear_model', linear_model.LinearRegression())])
    model.fit(X_train, y_train)
    return model.predict(X_test)

tfidf_lr = Regressor(dataset=dataset1,
                     estimator=tfidf_lr_,
                     name='tfidf_lr')

# ExtraTrees на NMF
rf = Regressor(dataset=dataset2, 
                     estimator=ensemble.ExtraTreesRegressor,
                     parameters={'n_estimators': 100, 'n_jobs': -1},
                     name='rf')

# Stack two models
# Returns new dataset with out-of-fold predictions
meta_pipeline = ModelsPipeline(lr, tfidf_lr, rf)
stack_ds = meta_pipeline.blend(proportion=0.2,seed=111)

# Train LinearRegression on stacked data (second stage)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
results = stacker.predict()

print rmse(y[itest], results)

### данные для отправки

In [ ]:
urls_test_df = pd.read_csv('data/url_domain_test.csv', header=None, delimiter='\t')
urls_test_df.columns = ['id', 'url', 'count']
urls_test_df.id.nunique()

In [ ]:
urls_test_df.head()

In [ ]:
sep_tmp = urls_test_df
sep_tmp['url'] = sep_tmp['url'].astype('str')
sep_tmp['space'] = ' '
sep_tmp['array_of_url'] = (sep_tmp['url'] + sep_tmp['space']).apply(lambda x: re.findall(r"[\w']+",x)) * sep_tmp['count']
sep_tmp.drop(['space'], axis = 1, inplace=True)
sep_tmp.head()

In [ ]:
TEST_URLS = pd.DataFrame(sep_tmp.groupby('id').array_of_url.apply(lambda x: x.tolist()))

In [ ]:
TEST_URLS['list'] = TEST_URLS.array_of_url.apply(sep_f)
TEST_URLS.drop(['array_of_url'], axis = 1, inplace=True)

In [ ]:
TEST_URLS['id'] = TEST_URLS.index
TEST_URLS.index = range(len(TEST_URLS))
TEST_URLS.columns = ['urls', 'id']
TEST_URLS.head()

In [ ]:
X_test= tfidf.transform(TEST_URLS.urls.values)

In [ ]:
lin_reg = linear_model.SGDRegressor(n_iter=750, alpha=0.000008)
lin_reg.fit(sep_X, y)

In [ ]:
y_pred = lin_reg.predict(X_test)

In [ ]:
TEST_URLS['age'] = y_pred

In [ ]:
TEST_URLS.head()

In [ ]:
TEST_URLS = TEST_URLS[['id', 'age']]
TEST_URLS.columns = ['Id', 'age']

In [ ]:
mean_array = TEST_URLS.age.mean()

In [ ]:
mean_array

In [ ]:
mean_array*np.ones(10)

In [ ]:
random_sol = pd.read_csv('data/sample_submission.csv')
random_sol.head()

In [ ]:
miss_idx = set(random_sol.Id.values) - set(TEST_URLS.Id.values)
miss_df = pd.DataFrame(zip(list(miss_idx), mean_array*np.ones(len(miss_idx))))
miss_df.columns = ['Id', 'age']

In [ ]:
TEST_URLS = TEST_URLS.append(miss_df, ignore_index=True)

In [ ]:
TEST_URLS.to_csv('my_solution.csv', index=False)

In [ ]:
TEST_URLS.shape

In [ ]:
print urls_test_df.shape
print TEST_URLS.shape

In [ ]:
print urls_test_df.id.nunique()
print TEST_URLS.id.nunique()